In [ ]:
#tf.compat.v1.disable_eager_execution()
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
import os

In [1]:
import os
import glob

import cv2
import numpy as np
import tensorflow as tf
from tensorboard.plugins import projector
import csv
#import tensorflow as tf
import tensorflow_hub as hub

In [2]:
LOG_DIR = "/Users/denisdunn/Desktop/logs"
IMAGES_DIR = "/Users/denisdunn/Desktop/unsupervised_images"
IMAGE_SIZE = (64, 64)
SPRITES_FILE = "sprites.png"
SPRITES_PATH = os.path.join(LOG_DIR, SPRITES_FILE)
FEATURE_VECTORS = "feature_vectors.npy"
METADATA_FILE = "metadata.tsv"
METADATA_PATH = os.path.join(LOG_DIR, METADATA_FILE)
CHECKPOINT_FILE = os.path.join(LOG_DIR, "features.ckpt")

# Max sprite size is 8192 x 8192 so this max samples makes visualization easy
MAX_NUMBER_SAMPLES = 8191

In [3]:
def create_sprite(data):
    """
    Tile images into sprite image. 
    Add any necessary padding
    """
    
    # For B&W or greyscale images
    if len(data.shape) == 3:
        data = np.tile(data[...,np.newaxis], (1,1,1,3))

    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0), (0, 0), (0, 0))
    data = np.pad(data, padding, mode='constant',
            constant_values=0)
    
    # Tile images into sprite
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3, 4))
    # print(data.shape) => (n, image_height, n, image_width, 3)
    
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    # print(data.shape) => (n * image_height, n * image_width, 3) 
    return data


In [ ]:
# Create sprite image

image_files = glob.glob(os.path.join(IMAGES_DIR, "*.png"))

img_data = []
for img in image_files[:MAX_NUMBER_SAMPLES]:
    input_img = cv2.imread(img)
    input_img_resize = cv2.resize(input_img, IMAGE_SIZE) 
    img_data.append(input_img_resize)
img_data = np.array(img_data)

sprite = create_sprite(img_data)
cv2.imwrite(SPRITES_PATH, sprite)

In [ ]:

# Create metadata, configure for tensorboard embedding

# Create metadata
# Can include class data in here if interested / have available
with open(METADATA_PATH, 'w+') as wrf:
    wrf.write("\n".join([str(a) for a,i in enumerate(image_files[:MAX_NUMBER_SAMPLES])]))

feature_vectors = np.load(FEATURE_VECTORS)

features = tf.Variable(feature_vectors[:MAX_NUMBER_SAMPLES], name='features')

# Write summaries for tensorboard
with tf.Session() as sess:
    saver = tf.train.Saver([features])

    sess.run(features.initializer)
    saver.save(sess, CHECKPOINT_FILE)

    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = features.name
    embedding.metadata_path = METADATA_FILE

    # This adds the sprite images
    embedding.sprite.image_path = SPRITES_FILE
    embedding.sprite.single_image_dim.extend(IMAGE_SIZE)
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)

In [4]:
image_file=os.listdir('/Users/denisdunn/Desktop/unsupervised_images')

In [5]:
image_file.remove('.DS_Store')

In [ ]:
tf.compat.v1.train.input_producer(
    input_tensor, element_shape=None, num_epochs=None, shuffle=True, seed=None,
    capacity=32, shared_name=None, summary_name=None, name=None, cancel_op=None
)
tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs). If shuffle=False, omit the .shuffle(...).


In [45]:

#iterator = iter(dataset)
module = hub.load("https://tfhub.dev/google/imagenet/mobilenet_v2_100_96/feature_vector/4")
height, width = (224,224)#hub.get_expected_image_size(module)
ch = 3
batch_size = 10

image_list = tf.convert_to_tensor(['/Users/denisdunn/Desktop/unsupervised_images/{}'.format(file) for file in range(len(image_file))])
#input_queue = tf.train.slice_input_producer([image_list], num_epochs=1, shuffle=False)
#input_queue = tf.compat.v1.train.slice_input_producer([image_list], num_epochs=1, shuffle=False)
input_queue=tf.data.Dataset.from_tensor_slices(image_list)
#input_queue=tf.data.Dataset.from_tensor_slices(image_list).shuffle(tf.shape(image_list, out_type=tf.int64)[0]).repeat(1)
#image_bytes = tf.io.read_file(input_queue)
#image = tf.io.decode_image(input_queue, channels=ch)
#image = tf.compat.v1.image.resize_bilinear(input_queue, [height, width])
#image = tf.reshape(image, tf.stack([height, width, ch]))
#images = tf.train(input_queue, batch_size=batch_size, allow_smaller_final_batch=True)
features = module(input_queue)

with open('feature_vecs.tsv', 'w') as fw:
    csv_writer = csv.writer(fw, delimiter='\t')
    with tf.Session() as sess:
        sess.run([tf.local_variables_initializer(),
                  tf.global_variables_initializer()])
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)

        try:
            while not coord.should_stop():
                fvecs = sess.run(features)
                csv_writer.writerows(fvecs)
        except:
            pass
        finally:
            coord.request_stop()
            coord.join(threads)

ValueError: Could not find matching function to call loaded from the SavedModel. Got:
  Positional arguments (4 total):
    * <_VariantDataset shapes: (), types: tf.string>
    * False
    * False
    * 0.99
  Keyword arguments: {}

Expected these arguments to match one of the following 4 option(s):

Option 1:
  Positional arguments (4 total):
    * TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name='inputs')
    * False
    * False
    * TensorSpec(shape=(), dtype=tf.float32, name='batch_norm_momentum')
  Keyword arguments: {}

Option 2:
  Positional arguments (4 total):
    * TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name='inputs')
    * False
    * True
    * TensorSpec(shape=(), dtype=tf.float32, name='batch_norm_momentum')
  Keyword arguments: {}

Option 3:
  Positional arguments (4 total):
    * TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name='inputs')
    * True
    * True
    * TensorSpec(shape=(), dtype=tf.float32, name='batch_norm_momentum')
  K

In [46]:
pwd

'/Users/denisdunn'